<a href="https://colab.research.google.com/github/cncoats7/training/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installations

In [ ]:
import os
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import PALChain

from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.chains import SimpleSequentialChain, SequentialChain
from langchain.memory import SimpleMemory


## Set API keys

In [ ]:

os.environ["OPENAI_API_KEY"] = "..."
os.environ["SERPAPI_API_KEY"] = "..."

## Prompt Creation

In [ ]:

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

print(prompt.format(product="podcast player"))

## Generic Chain

In [ ]:
# Usecase 1: Get predictions from existing model (LLM Primitive)
llm = OpenAI(
          model_name="text-davinci-003", # default model
          temperature=0.9) #temperature dictates how whacky the output should be
llmchain = LLMChain(llm=llm, prompt=prompt)
llmchain.run("podcast player")

In [ ]:
chatopenai = ChatOpenAI(model_name="gpt-3.5-turbo")
llmchain_chat = LLMChain(llm=chatopenai, prompt=prompt)
llmchain_chat.run("podcast player")

## Utility Chains

In [ ]:
palchain = PALChain.from_math_prompt(llm=llm, verbose=True)
palchain.run("If my age is half of my dad's age and he is going to be 60 next year, what is my current age?")

In [ ]:
# checkout default prompt for the chain
print(palchain.prompt.template)

## Agents & Tools

In [ ]:
llm = OpenAI(temperature=0)
tools = load_tools(["pal-math"], llm=llm)

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

agent.run("If my age is half of my dad's age and he is going to be 60 next year, what is my current age?")

In [ ]:
# check the default prompt for the chain
print(agent.agent.llm_chain.prompt.template)

In [ ]:
# make the question more complex
agent.run("My age is half of my dad's age. Next year he is going to be same age as Demi Moore. What is my current age?")

As you can see the agent is not using the most recent age information for Demi Moore. Let's fix this!

In [ ]:
# Add a new tool for the agent to use
tools = load_tools(["pal-math", "serpapi"], llm=llm)
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)
agent.run("My age is half of my dad's age. Next year he is going to be same age as Demi Moore. What is my current age?")

In [ ]:
# let's try one more tool, this time an API-based tool
tools = load_tools(["podcast-api"], llm=llm, listen_api_key="...")
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

agent.run("Show me episodes for money saving tips.")

Note1: There is a known error with using this API where you might see, `openai.error.InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested XXX tokens (XX in your prompt; XX for the completion). Please reduce your prompt; or completion length.`
This happens when the response returned by the API might be too big.

## Create chains

### Simple Sequential Chains
Each entity in chain must have single input and output

In [ ]:
# Chain1 - solve math problem, get the age
chain_one = agent

In [ ]:
# Chain2 - suggest age-appropriate gift
template = """You are a gift recommender. Given a person's age,\n
 it is your job to suggest an appropriate gift for them.

Person Age:
{age}
Suggest gift:"""
prompt_template = PromptTemplate(input_variables=["age"], template=template)
chain_two = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
# create simple sequential chains using the above
overall_chain = SimpleSequentialChain(
                  chains=[chain_one, chain_two],
                  verbose=True)

In [ ]:
# test the chain using a complex math problem
question = "If my age is half of my dad's age and he is going to be 60 next year, what is my current age?"
overall_chain.run(question)

### Sequential Chains
Each entity in chain can have multiple inputs/outputs

In [ ]:
template = """You are a gift recommender. Given a person's age,\n
 it is your job to suggest an appropriate gift for them. If age is under 10,\n
 the gift should cost no more than {budget} otherwise it should cost atleast 10 times {budget}.

Person Age:
{output}
Suggest gift:"""
prompt_template = PromptTemplate(input_variables=["output", "budget"], template=template)
chain_two = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
overall_chain = SequentialChain(
                input_variables=["input"],
                memory=SimpleMemory(memories={"budget": "100 GBP"}),
                chains=[agent, chain_two],
                verbose=True)

In [ ]:
overall_chain.run("If my age is half of my dad's age and he is going to be 60 next year, what is my current age?")

In [ ]:
# check the prompt template for the agent
print(agent.agent.llm_chain.prompt.template)

In [ ]:
# check the input variables for the agent
print(agent.agent.llm_chain.prompt.input_variables)